<a href="https://github.com/timeseriesAI/tsai-rs" target="_parent"><img src="https://img.shields.io/badge/tsai--rs-Time%20Series%20AI%20in%20Rust-blue" alt="tsai-rs"/></a>

# Time Series Architecture Comparison with tsai-rs

This notebook compares different deep learning architectures for time series classification using **tsai-rs**.

## Purpose

tsai-rs provides multiple state-of-the-art architectures:

1. **InceptionTimePlus** - CNN-based with multi-scale convolutions
2. **ResNetPlus** - Deep residual networks
3. **PatchTST** - Transformer with patching
4. **TST** - Time Series Transformer
5. **RNNPlus** - LSTM/GRU based models
6. **MiniRocket** - Fast feature extraction with linear classifier

This notebook shows how to configure each architecture.

## Install tsai-rs

```bash
cd crates/tsai_python
maturin develop --release
```

## Import Libraries

In [ ]:
import tsai_rs
import numpy as np

print(f"tsai-rs version: {tsai_rs.version()}")
tsai_rs.my_setup()

## Load Sample Data

In [ ]:
# Load a multivariate dataset
dsid = 'NATOPS'
X_train, y_train, X_test, y_test = tsai_rs.get_UCR_data(dsid, return_split=True)

# Get data dimensions
n_samples = X_train.shape[0]
n_vars = X_train.shape[1]
seq_len = X_train.shape[2]
n_classes = len(np.unique(y_train))

print(f"Dataset: {dsid}")
print(f"Train samples: {n_samples}")
print(f"Test samples: {X_test.shape[0]}")
print(f"Variables: {n_vars}")
print(f"Sequence length: {seq_len}")
print(f"Classes: {n_classes}")
print(f"Class labels: {np.unique(y_train)}")

## Standardize Data

In [ ]:
X_train_std = tsai_rs.ts_standardize(X_train.astype(np.float32), by_sample=True)
X_test_std = tsai_rs.ts_standardize(X_test.astype(np.float32), by_sample=True)

print(f"Standardized data shape: {X_train_std.shape}")

## Architecture 1: InceptionTimePlus

InceptionTime is a CNN-based architecture that uses inception modules with multi-scale convolutions. It's fast, accurate, and works well on most datasets.

In [ ]:
# Default configuration
inception_default = tsai_rs.InceptionTimePlusConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes
)
print(f"InceptionTimePlus (default): {inception_default}")

In [ ]:
# Custom configuration with more filters
inception_large = tsai_rs.InceptionTimePlusConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes,
    nf=64,        # Number of filters (default: 32)
    depth=8       # Number of inception modules (default: 6)
)
print(f"InceptionTimePlus (large): {inception_large}")

In [ ]:
# With dropout for regularization
inception_dropout = tsai_rs.InceptionTimePlusConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes,
    nf=32,
    fc_dropout=0.3  # Dropout in final classifier
)
print(f"InceptionTimePlus (with dropout): {inception_dropout}")

## Architecture 2: ResNetPlus

ResNet uses residual connections to train deeper networks effectively.

In [ ]:
# Default ResNet configuration
resnet_default = tsai_rs.ResNetPlusConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes
)
print(f"ResNetPlus (default): {resnet_default}")

In [ ]:
# Deeper ResNet
resnet_deep = tsai_rs.ResNetPlusConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes,
    nf=128  # More filters
)
print(f"ResNetPlus (deep): {resnet_deep}")

## Architecture 3: PatchTST

PatchTST applies the transformer architecture to time series by dividing the sequence into patches, similar to Vision Transformer (ViT) for images.

In [ ]:
# Default PatchTST configuration
patchtst_default = tsai_rs.PatchTSTConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes,
    patch_len=8,      # Length of each patch
    stride=8          # Stride between patches
)
print(f"PatchTST (default): {patchtst_default}")

In [ ]:
# Larger PatchTST with more attention heads
patchtst_large = tsai_rs.PatchTSTConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes,
    patch_len=16,
    stride=8,          # Overlapping patches
    d_model=128,
    n_heads=8,
    n_layers=4,
    d_ff=256,
    dropout=0.1
)
print(f"PatchTST (large): {patchtst_large}")

## Architecture 4: TST (Time Series Transformer)

TST is a pure transformer architecture for time series that processes each timestep as a token.

In [ ]:
# Default TST configuration
tst_default = tsai_rs.TSTConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes
)
print(f"TST (default): {tst_default}")

In [ ]:
# Custom TST with specific hyperparameters
tst_custom = tsai_rs.TSTConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes,
    d_model=64,       # Model dimension
    n_heads=4,        # Number of attention heads
    n_layers=3,       # Number of transformer layers
    d_ff=128,         # Feed-forward dimension
    dropout=0.1,      # Dropout rate
    fc_dropout=0.2    # Final classifier dropout
)
print(f"TST (custom): {tst_custom}")

## Architecture 5: RNNPlus

RNNPlus provides LSTM and GRU based architectures, which are effective for sequential data.

In [ ]:
# LSTM configuration
lstm_config = tsai_rs.RNNPlusConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes,
    rnn_type='lstm',
    hidden_size=64,
    n_layers=2,
    bidirectional=True,
    dropout=0.2
)
print(f"LSTM: {lstm_config}")

In [ ]:
# GRU configuration
gru_config = tsai_rs.RNNPlusConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes,
    rnn_type='gru',
    hidden_size=128,
    n_layers=3,
    bidirectional=True,
    dropout=0.3
)
print(f"GRU: {gru_config}")

In [ ]:
# Unidirectional RNN (for real-time/streaming applications)
rnn_uni = tsai_rs.RNNPlusConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes,
    rnn_type='lstm',
    hidden_size=64,
    n_layers=2,
    bidirectional=False  # One direction only
)
print(f"Unidirectional LSTM: {rnn_uni}")

## Architecture 6: MiniRocket

MiniRocket is a feature extraction method that transforms time series into fixed-length feature vectors, which are then classified using a linear model. It's extremely fast while maintaining competitive accuracy.

In [ ]:
# Default MiniRocket configuration
minirocket_default = tsai_rs.MiniRocketConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes
)
print(f"MiniRocket (default): {minirocket_default}")

In [ ]:
# MiniRocket with more features
minirocket_large = tsai_rs.MiniRocketConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes,
    n_features=10000   # More features for potentially better accuracy
)
print(f"MiniRocket (large): {minirocket_large}")

## Architecture Comparison Table

In [ ]:
architectures = [
    ('InceptionTimePlus', inception_default, 'CNN', 'Multi-scale convolutions'),
    ('ResNetPlus', resnet_default, 'CNN', 'Residual connections'),
    ('PatchTST', patchtst_default, 'Transformer', 'Patching + attention'),
    ('TST', tst_default, 'Transformer', 'Pure attention'),
    ('RNNPlus (LSTM)', lstm_config, 'RNN', 'Sequential processing'),
    ('RNNPlus (GRU)', gru_config, 'RNN', 'Gated units'),
    ('MiniRocket', minirocket_default, 'Feature', 'Random kernels + linear'),
]

print(f"{'Architecture':<20} {'Type':<12} {'Description':<30}")
print("-" * 65)
for name, config, arch_type, desc in architectures:
    print(f"{name:<20} {arch_type:<12} {desc:<30}")

## Choosing the Right Architecture

| Use Case | Recommended Architecture |
|----------|-------------------------|
| General purpose, fast training | InceptionTimePlus |
| Very deep networks | ResNetPlus |
| Long sequences | PatchTST |
| Capturing long-range dependencies | TST, PatchTST |
| Streaming/real-time | RNNPlus (unidirectional) |
| Very fast training needed | MiniRocket |
| Small datasets | MiniRocket, InceptionTimePlus |
| Large datasets | Any (try PatchTST/TST) |

## Training Configuration

In [ ]:
# Common learner configuration
learner_config = tsai_rs.LearnerConfig(
    lr=1e-3,
    weight_decay=0.01,
    grad_clip=1.0
)
print(f"Learner config: {learner_config}")

In [ ]:
# One-cycle learning rate scheduler
n_epochs = 25
batch_size = 32
steps_per_epoch = (n_samples + batch_size - 1) // batch_size
total_steps = n_epochs * steps_per_epoch

scheduler = tsai_rs.OneCycleLR.simple(max_lr=1e-3, total_steps=total_steps)
print(f"OneCycleLR scheduler: {scheduler}")

## Benchmark Setup Example

In [ ]:
def create_configs_for_dataset(X, y):
    """Create all architecture configs for a given dataset."""
    n_vars = X.shape[1]
    seq_len = X.shape[2]
    n_classes = len(np.unique(y))
    
    configs = {
        'InceptionTimePlus': tsai_rs.InceptionTimePlusConfig(
            n_vars=n_vars, seq_len=seq_len, n_classes=n_classes
        ),
        'ResNetPlus': tsai_rs.ResNetPlusConfig(
            n_vars=n_vars, seq_len=seq_len, n_classes=n_classes
        ),
        'PatchTST': tsai_rs.PatchTSTConfig.for_classification(
            n_vars=n_vars, seq_len=seq_len, n_classes=n_classes
        ),
        'TST': tsai_rs.TSTConfig(
            n_vars=n_vars, seq_len=seq_len, n_classes=n_classes,
            d_model=64, n_heads=4, n_layers=2
        ),
        'LSTM': tsai_rs.RNNPlusConfig(
            n_vars=n_vars, seq_len=seq_len, n_classes=n_classes,
            rnn_type='lstm', hidden_size=64, n_layers=2, bidirectional=True
        ),
        'MiniRocket': tsai_rs.MiniRocketConfig(
            n_vars=n_vars, seq_len=seq_len, n_classes=n_classes
        ),
    }
    return configs

# Create configs for our dataset
configs = create_configs_for_dataset(X_train, y_train)

print("Architecture configurations for NATOPS dataset:")
print("=" * 50)
for name, config in configs.items():
    print(f"\n{name}:")
    print(f"  {config}")

## Test on Multiple Datasets

In [ ]:
datasets = ['ECG200', 'GunPoint', 'FordA', 'Wafer', 'NATOPS']

print(f"{'Dataset':<15} {'Train':<8} {'Test':<8} {'Vars':<6} {'Len':<8} {'Classes':<8}")
print("-" * 60)

for dsid in datasets:
    try:
        X_train, y_train, X_test, y_test = tsai_rs.get_UCR_data(dsid, return_split=True)
        n_classes = len(np.unique(y_train))
        
        print(f"{dsid:<15} {X_train.shape[0]:<8} {X_test.shape[0]:<8} "
              f"{X_train.shape[1]:<6} {X_train.shape[2]:<8} {n_classes:<8}")
        
    except Exception as e:
        print(f"{dsid:<15} Error: {e}")

## Summary

This notebook demonstrated the architecture options in tsai-rs:

### CNN-based
- **InceptionTimePlus**: Multi-scale convolutions, fast and accurate
- **ResNetPlus**: Deep residual networks

### Transformer-based
- **PatchTST**: Patches + transformer, good for long sequences
- **TST**: Pure transformer, captures long-range dependencies

### RNN-based
- **RNNPlus**: LSTM/GRU, good for sequential patterns

### Feature-based
- **MiniRocket**: Extremely fast, random kernel features

### Key Configurations
All architectures share common parameters:
- `n_vars`: Number of input variables
- `seq_len`: Sequence length
- `n_classes`: Number of output classes

Architecture-specific parameters allow fine-tuning for specific datasets.

In [ ]:
# Quick reference: creating configs
n_vars, seq_len, n_classes = 24, 51, 6

print("Quick reference for creating configs:")
print("=" * 50)
print(f"\nInceptionTimePlus:")
print(f"  tsai_rs.InceptionTimePlusConfig(n_vars={n_vars}, seq_len={seq_len}, n_classes={n_classes})")
print(f"\nResNetPlus:")
print(f"  tsai_rs.ResNetPlusConfig(n_vars={n_vars}, seq_len={seq_len}, n_classes={n_classes})")
print(f"\nPatchTST:")
print(f"  tsai_rs.PatchTSTConfig.for_classification(n_vars={n_vars}, seq_len={seq_len}, n_classes={n_classes})")
print(f"\nTST:")
print(f"  tsai_rs.TSTConfig(n_vars={n_vars}, seq_len={seq_len}, n_classes={n_classes}, d_model=64, n_heads=4)")
print(f"\nRNNPlus:")
print(f"  tsai_rs.RNNPlusConfig(n_vars={n_vars}, seq_len={seq_len}, n_classes={n_classes}, rnn_type='lstm')")
print(f"\nMiniRocket:")
print(f"  tsai_rs.MiniRocketConfig(n_vars={n_vars}, seq_len={seq_len}, n_classes={n_classes})")